In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
MAX_LEN=500
MAX_WORDS = 1000

In [ ]:
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

In [ ]:
train_labels = np.where(df_train['label'] == 'spam', 1, 0)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['message'])
word_index = tokenizer.word_index

In [ ]:
def encode_text(text):
    tokens = tokenizer.texts_to_sequences([text])
    return pad_sequences(tokens, maxlen=MAX_LEN)[0]

In [ ]:
train_labels = np.where(df_train['label'] == 'spam', 1, 0)
train_messages = np.array([encode_text(text) for text in df_train['message']])

In [ ]:
ham_count = np.sum(train_labels == 0)
spam_count = np.sum(train_labels == 1)
print(f"Ham count: {ham_count}, Spam count: {spam_count}")

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(MAX_WORDS, 50),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])

In [ ]:
model.fit(train_messages, train_labels, epochs=10, validation_split=0.2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LEN)

    prediction = model.predict(padded_sequence)[0]

    label = 'ham' if prediction < 0.5 else 'spam'
    print(prediction[0],label)
    return [prediction[0], label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
